<a href="https://colab.research.google.com/github/timosachsenberg/EuBIC2026/blob/main/notebooks/EUBIC_Task1_Peaks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies (for Google Colab)
!pip install -q pyopenms>=3.5.0 pyopenms-viz>=1.0.0

# Notebook 1 – From Proteins to Spectra: Digestion & First Look at LC–MS Data

In this notebook we will:

1. **Get acquainted with `pyopenms` and enzymatic digestion**  
   - Load a small **FASTA file** (a text format for storing protein/nucleotide sequences) with a few protein sequences  
   - Perform an **in-silico** (computational, as opposed to in a wet lab) **Trypsin** digestion  
   - Generate realistic bottom-up **peptides** with length and missed-cleavage constraints  

2. **Take a first look at typical LC–MS data**  
   - Load an **mzML file** (an open XML format for mass spectrometry data)  
   - Visualize the data as a 2D **peak map (heatmap)**  
   - Extract and plot a **single MS1 spectrum**  

3. **Zoom into isotope patterns**  
   - Find an **isotope pattern** of an analyte  
   - Estimate the **charge state** and **mass** of the analyte  

4. **Compute the Total Ion Current (TIC)**  
   - Calculate the **TIC chromatogram** as the sum of all ion intensities over time  
   - Interpret the axes and what the TIC tells us about the LC–MS run

---

<details>
<summary><b>Quick Reference: Key Terms Used in This Notebook</b></summary>

| Term | Definition | Example |
|------|------------|---------|
| **m/z** | Mass-to-charge ratio. The mass of an ion divided by its charge. | A peptide with mass 1000 Da and charge +2 has m/z = 500 |
| **RT** | Retention time - when an analyte elutes from the LC column (seconds) | RT = 1200 s means the peptide eluted at 20 minutes |
| **MS1** | Survey scan measuring intact peptide masses | Used to detect which peptides are present |
| **Isotope pattern** | Group of peaks from the same molecule with different isotopes | Peaks spaced ~0.5 m/z apart for a +2 ion |
| **TIC** | Total Ion Current - sum of all ion intensities at each time point | Shows when analytes elute |
| **FASTA** | Text file format for protein/DNA sequences | Each entry has a header (>) and sequence |
| **mzML** | Standard XML format for mass spectrometry data | Contains spectra with m/z and intensity arrays |
| **Da (Dalton)** | Unit of molecular mass (same as unified atomic mass unit, u) | Alanine residue = ~71 Da |

</details>

In [ ]:
%matplotlib inline
import pyopenms as oms
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
print("pyOpenMS version:", oms.__version__)

pyOpenMS version: 3.5.0


## 1. Enzymatic digestion with Trypsin (in-silico)

**Aim of this task**

We want to get familiar with `pyopenms` and simulate a simple **bottom-up proteomics** scenario:

- We start from a small **FASTA file** containing two human proteins from the UPS1 standard (Complement C5 and Pro-epidermal growth factor).
- We apply a **Trypsin** digestion:
  - Trypsin cleaves **after K (Lys) and R (Arg)**, except when followed by **P (Proline)**.
- We allow up to **2 missed cleavages**, because the enzyme is not perfect.
- We only keep peptides with length between **7 and 40 amino acids**, so that they are realistic for LC–MS/MS.

At the end, we will:
- Print the protein entries from the FASTA file
- Generate in-silico peptides for each protein
- Count and print the **total number of peptides**.

<details>
<summary><b>New to Python? Understanding the Code Patterns</b></summary>

This notebook uses several Python patterns you'll see repeatedly:

**1. Loading data into objects:**
```python
fasta = oms.FASTAFile()      # Create an empty container object
entries = []                  # Create an empty list to store results
fasta.load("file.fasta", entries)  # Load fills the list with data
```

**2. Iterating over collections:**
```python
for entry in entries:         # Loop through each item
    print(entry.identifier)   # Access properties with dot notation
```

**3. List operations:**
```python
peptides = []                 # Empty list
peptides[:5]                  # First 5 items (slicing)
len(peptides)                 # Count items
```

**4. f-strings for formatting:**
```python
print(f"Found {len(peptides)} peptides")  # Variables inside {}
```

</details>

<details>
<summary><b>Deep Dive: Why Does Trypsin Cleave After K and R?</b></summary>

**The Biochemistry Behind Trypsin Specificity**

Trypsin is a serine protease with a negatively charged binding pocket (aspartate residue at position 189). This pocket attracts the positively charged side chains of:

- **Lysine (K)**: Has a positively charged amino group (-NH3+)
- **Arginine (R)**: Has a positively charged guanidinium group

```
Trypsin cleavage sites:

    ...Ala─Lys─┃─Glu...     Cleaves after Lys (K)
              ↓
    ...Ala─Arg─┃─Asp...     Cleaves after Arg (R)
              ↓
    ...Ala─Lys─╳─Pro...     Does NOT cleave before Pro (P)
```

**Why not before Proline?**

Proline's cyclic structure creates a rigid kink in the peptide backbone that prevents proper positioning in trypsin's active site. This is called the "proline rule."

**Why is this useful for proteomics?**

Tryptic peptides have predictable properties:
- End with K or R (except C-terminus) → good for MS/MS fragmentation
- Average length ~10-15 amino acids → ideal for LC-MS
- Generate charge states +2 to +4 → optimal for detection

</details>

In [ ]:
# Download the FASTA file from the course repository (2 human UPS proteins)
!wget -q -O "two_ups_proteins.fasta" https://raw.githubusercontent.com/timosachsenberg/EuBIC2026/main/data/two_ups_proteins.fasta

fasta = oms.FASTAFile()
entries = []
fasta.load("two_ups_proteins.fasta", entries)

print(f"Loaded {len(entries)} FASTA entries\n")

for i, entry in enumerate(entries):
    print(f"Entry {i}")
    print("  ID:     ", entry.identifier)
    print("  Desc:   ", entry.description)
    print("  Length: ", len(entry.sequence), "aa")
    print("  Seq (first 60 aa):", entry.sequence[:60], "...\n")

Typically we only observe peptides of a certain size (about 7–40 amino acids) reliably. Very small ones are ambiguous and unstable; very large ones don’t ionize or fragment well.

In practice, proteolytic enzymes do not cleave perfectly. These missed cleavages produce longer peptides that still contain internal cleavage sites. Explore how allowing missed cleavages increases the number of generated peptides.



```
# Change this missed cleavages setting to see influence of missed cleavages on total number of peptides
digestion.setMissedCleavages()
```




Accounting for both missed cleavages and length restrictions will generate realistic peptide sequences that we could in principle measure by mass spectrometry.

In [ ]:
# Create a ProteaseDigestion object and configure Trypsin
digestion = oms.ProteaseDigestion()
digestion.setEnzyme("Trypsin")       # use Trypsin
digestion.setMissedCleavages(2)     # here allow up to 2 missed cleavages

# min and max length of peptide allow
min_len = 7
max_len = 40

# iterate over entries in fasta
for entry in entries:
    protein_seq = oms.AASequence.fromString(entry.sequence) # convert to AASequence object
    peptides = [] # empty list to store digested peptides list
    digestion.digest(protein_seq, peptides, min_len, max_len) # do the digestion

    print("\nNumber of in-silico peptides of protein", entry.identifier, ":", len(peptides))

    # Show a few example peptides
    print("\nExample peptides:")
    for pep in peptides[:5]:
      print(f"    {pep.toString()} (length {pep.size()})")



Number of in-silico peptides of protein P68509|1433F_BOVIN : 67

Example peptides:
    LAEQAER (length 7)
    YDDMASAMK (length 9)
    AVTELNEPLSNEDR (length 14)
    NLLSVAYK (length 8)
    VISSIEQK (length 8)

Number of in-silico peptides of protein Q9CQV8|1433B_MOUSE : 66

Example peptides:
    LAEQAER (length 7)
    YDDMAAAMK (length 9)
    AVTEQGHELSNEER (length 14)
    NLLSVAYK (length 8)
    VISSIEQK (length 8)


---
### Exercise 1: Explore Missed Cleavages

**Predict first, then verify!** This is how scientists think.

1. **Prediction**: If we change `missed_cleavages` from 2 to 0, will we get MORE or FEWER peptides?
2. **Why?** Write down your reasoning before running the code.
3. **Verify**: Modify the code above and re-run. Were you correct?

<details>
<summary><b>Click to reveal the answer</b></summary>

**Answer**: FEWER peptides.

With 0 missed cleavages, we only get peptides from "perfect" cleavage - every K and R is cut. With 2 missed cleavages, we also get peptides that span 1 or 2 cleavage sites.

**Example**: The sequence `LAEQAERYDDMASAMK` contains one internal K.
- With 0 missed cleavages: `LAEQAER` and `YDDMASAMK` (2 peptides)
- With 1 missed cleavage: Also `LAEQAERYDDMASAMK` (3 peptides total)

Typical results:
- 0 missed cleavages: ~40 peptides per protein
- 1 missed cleavage: ~55 peptides per protein  
- 2 missed cleavages: ~65 peptides per protein

</details>

---

## 2. First look at MS1 data: peak map and single spectrum

**Aim of this task**

We now want to look at **real mass spectrometry data**.

- We load an mzML file containing **MS1 spectra**.
- We visualize the data as a 2D **peak map**:
  - **x-axis**: retention time (RT, usually in seconds or minutes)
  - **y-axis**: mass-to-charge ratio (**m/z**)
  - **color**: signal **intensity** at that RT–m/z position

This helps us get a global overview of the LC–MS run.

Then:

- We extract **one single MS1 spectrum**.
- We plot this spectrum as **intensity vs. m/z**.
- This allows us to see how a single snapshot of the LC–MS run looks like.


In [ ]:
# Download the mzML file from the course repository (5-minute subset of UPS1 data)
!wget -q -O "UPS1_5min.mzML" https://raw.githubusercontent.com/timosachsenberg/EuBIC2026/main/data/UPS1_5min.mzML

# initialize MSExperiment object for 1D peak data
exp = oms.MSExperiment()
oms.MzMLFile().load("UPS1_5min.mzML", exp)
print(f"Loaded {exp.size()} spectra")

In [ ]:
# checkout pyOpenMS tutorial https://pyopenms.readthedocs.io/en/latest/user_guide/ms_data.html
def plot_spectra_2D(exp, ms_level=1, marker_size=5):
    fig = plt.figure(figsize=(12,8))
    exp.updateRanges()
    # extract each spectra from experiment
    for spec in exp:
        if spec.getMSLevel() == ms_level: # just take MS1 spectras
            mz, intensity = spec.get_peaks() # get peaks: mz corrsponding intensity
            p = intensity.argsort()  # sort by intensity to plot highest on top
            rt = np.full([mz.shape[0]], spec.getRT(), float) #Each spectrum has a single RT, but many m/z points, you broadcast the RT to match the length of the m/z array.
            plt.scatter(
                rt,
                mz[p],
                c=intensity[p],
                cmap="afmhot_r",
                s=marker_size,
                norm=colors.LogNorm(
                    exp.getMinIntensity() + 1, exp.getMaxIntensity()
                ),
            )
    plt.clim(exp.getMinIntensity() + 1, exp.getMaxIntensity())
    plt.xlabel("retention time (s)")
    plt.ylabel("m/z")
    plt.colorbar()
    plt.show()  # slow for larger data sets

# plot the spectra 2D
plot_spectra_2D(exp)


<details>
<summary><b>Understanding the 2D Peak Map</b></summary>

**How to read this visualization:**

```
       High intensity (bright/yellow)
            ↓
    ┌──────────────────────────┐
m/z │  ·  ·  ··· ·    ·  ·    │ ← Each dot is a detected ion
    │    ···████···   ···     │ ← Bright regions = many ions
    │  ·  ·█████·  ·  ··█·    │
    │     ·███·        ·█·    │
    └──────────────────────────┘
          RT (time) →
            
            Low intensity (dark)
```

**What patterns to look for:**

1. **Horizontal streaks**: Same m/z over time = same ion eluting
2. **Vertical lines**: Many different m/z at one time = complex elution
3. **Bright spots**: High-abundance analytes
4. **Diagonal patterns**: Could indicate ion suppression or gradient effects

**The color scale uses logarithmic normalization** because MS intensities span many orders of magnitude (10 to 10,000,000+).

</details>

---

### Exercise 2: Interpret the 2D Peak Map

Look at the 2D peak map above and answer these questions:

1. At approximately what retention time do you see the most intense signals?
2. What m/z range contains most of the detected ions?
3. Do you notice any horizontal "streaks" that might indicate the same analyte eluting over time?

<details>
<summary><b>Click for discussion</b></summary>

**Observations from this data:**

1. **Retention time**: The data spans 2400-2700 seconds (~40-45 minutes). Look for regions with the brightest signals.

2. **m/z range**: Most peptide ions appear in the 400-800 m/z range, which is typical for tryptic peptides with charge states +2 to +3.

3. **Horizontal streaks**: Yes! Look for signals that persist across multiple scans at the same m/z. This is an analyte eluting over its chromatographic peak width.

**Why this matters**: Understanding these patterns helps you:
- Identify potential contaminants (signals at all RTs)
- Assess chromatography quality (peak width)
- Find regions to zoom into for analysis

</details>

---

## 2.1 Plotting a single MS1 spectrum

Now we zoom into a **single MS1 spectrum**.

- A spectrum is a set of **peaks** measured at one retention time:
  - Each peak has an m/z value and an intensity.
- We plot **intensity vs. m/z**.

Tasks:

- Choose an MS1 spectrum index.
- Extract its m/z and intensity arrays via `get_peaks()`.
- Plot the spectrum as a stem plot.
- Clearly label axes:
  - x-axis: m/z
  - y-axis: intensity

optional: plot the spectrum with `pyopenms-viz` using `plot_spectrum`

In [ ]:
# Take one mass spectrum from provided experiment data
MS1_spectrum = exp[0]  # By modifying the number, you can access the other spectrums
print("check spectrum level: ", MS1_spectrum.getMSLevel())

In [ ]:
# extract the peaks in form of mzs,intensities
mzs, intensities = MS1_spectrum.get_peaks()

print(f"Retention time: {MS1_spectrum.getRT():.1f} seconds "
      f"({MS1_spectrum.getRT()/60:.2f} minutes)")
print(f"Number of peaks: {len(mzs)}")

plt.figure(figsize=(12, 5))
plt.stem(mzs, intensities)#, use_line_collection=True)
plt.xlabel("m/z")
plt.ylabel("Intensity")
plt.title("Single MS1 spectrum (intensity vs. m/z)")
plt.tight_layout()
plt.show()

# Explanation:
# Each vertical line (stem) is a peak corresponding to ions with a certain m/z.
# The height of the line is the intensity, proportional to how many ions of that m/z were detected in this scan.

In [ ]:
# plot it with pyopenms-viz
# pyopenms-viz extends pandas DataFrame plotting
import pandas as pd
import pyopenms_viz  # registers the plotting backend

# convert spectrum to DataFrame using get_peaks()
mzs, intensities = MS1_spectrum.get_peaks()
spectrum_df = pd.DataFrame({'mz': mzs, 'intensity': intensities})
spectrum_df.plot(kind='spectrum', x='mz', y='intensity', backend='ms_plotly', width=900, height=400);

## 3. Isotope patterns: estimate charge and mass

In high-resolution MS1 spectra, analytes often appear as **isotope patterns**:

- Several peaks with the **same charge** but slightly different m/z values.
- The spacing between peaks is approximately **1 / z** m/z units, where *z* is the charge. Corresponding to an additional neutron.

We will:

1. **Zoom into a small m/z region** of one MS1 spectrum to visually find an isotope pattern.  
2. Visually measure the spacing between isotopic peaks to estimate the **charge z**.  
3. Estimate the **neutral mass** of the analyte:

m_neutral ≈ (m/z) * z - z * m_p


where \( m_p \) is the proton mass (~1.0073 Da).

<details>
<summary><b>Deep Dive: Why Do Isotope Patterns Exist?</b></summary>

**The Physics Behind Isotope Patterns**

Most elements have multiple stable isotopes with different numbers of neutrons:

| Element | Most abundant | Less abundant | Natural abundance |
|---------|--------------|---------------|-------------------|
| Carbon | 12C (6p, 6n) | 13C (6p, 7n) | 98.9% / 1.1% |
| Nitrogen | 14N | 15N | 99.6% / 0.4% |
| Oxygen | 16O | 17O, 18O | 99.8% / 0.04% / 0.2% |
| Sulfur | 32S | 33S, 34S | 95% / 0.8% / 4.2% |

**For a peptide with ~50 carbon atoms:**
- ~50% of molecules have all 12C (monoisotopic)
- ~35% have one 13C
- ~12% have two 13C
- etc.

```
Isotope Pattern for a +2 charged peptide:

Intensity
    │
100%│    ▓▓▓ ← Monoisotopic (M)
    │    ███
 75%│    ███  ▓▓▓ ← M+1 (one 13C)
    │    ███  ███
 50%│    ███  ███
    │    ███  ███  ▓▓▓ ← M+2 (two 13C)
 25%│    ███  ███  ███
    │    ███  ███  ███  ▓▓▓
    └────┴────┴────┴────┴────→ m/z
         |    |
         └─0.5─┘  (spacing = 1.0 Da / 2 charges)
```

**The Key Insight**: The spacing between isotope peaks in m/z directly tells you the charge state!
- Spacing = 1.0 m/z → charge = +1
- Spacing = 0.5 m/z → charge = +2
- Spacing = 0.33 m/z → charge = +3
- General formula: **charge = 1 / spacing**

</details>

In [ ]:
# Select again the same MS1 spectrum
#mzs, intensities = MS1_spectrum.get_peaks()

# Define an m/z window to zoom into.
# In a real training, participants could adjust these values interactively.
#mz_min_zoom = 645
#mz_max_zoom = 647

#mask = (mzs >= mz_min_zoom) & (mzs <= mz_max_zoom)
#mzs_zoom = mzs[mask]
#int_zoom = intensities[mask]

#plt.figure(figsize=(10, 4))
#plt.stem(mzs_zoom, int_zoom)#, use_line_collection=True)
#plt.xlabel("m/z")
#plt.ylabel("Intensity")
#plt.title(f"Zoomed MS1 spectrum: {mz_min_zoom}–{mz_max_zoom} m/z")
#plt.tight_layout()
#plt.show()

#print("Inspect the zoomed region to visually identify an isotope pattern.")
#print("Look for a group of peaks with nearly regular spacing.")


In [ ]:
# with pyopenms-viz - use interactive zoom to explore isotope patterns
import pandas as pd
import pyopenms_viz  # registers the plotting backend

# convert spectrum to DataFrame using get_peaks()
mzs, intensities = MS1_spectrum.get_peaks()
spectrum_df = pd.DataFrame({'mz': mzs, 'intensity': intensities})

# plot the full spectrum - zoom into 407-410 m/z to see the isotope pattern
spectrum_df.plot(kind='spectrum', x='mz', y='intensity', backend='ms_plotly',
                 width=900, height=400, title='Zoom into 407–410 m/z to see isotope pattern');

We can see a clear isotope pattern. The peak at ~407.76 m/z is the lightest peak in the pattern and likely the monoisotopic peak. The next peak (to the right) at ~408.26 m/z corresponds to the molecule with one additional neutron in one of its atoms. We don't know which atom in each molecule has the additional neutron—it could be any. This means that the many ions recorded to form this isotopic peak are likely a mixture of many isotopic variants of the molecule, each with an additional neutron. 

The spacing between the isotopic peaks is ~0.5 m/z. We know that a neutron adds ~1.0 Da (or u). If we observe a spacing of 0.5 m/z, it tells us that the charge z of the ion is +2.

---

### Exercise 3: Calculate Charge State and Neutral Mass

Using the isotope pattern at 407-410 m/z, complete these calculations:

**Part A: Determine the charge state**
1. The monoisotopic peak is at ~407.76 m/z
2. The next isotope peak (M+1) is at ~408.26 m/z  
3. What is the spacing? What charge state does this indicate?

**Part B: Calculate the neutral mass**

Use the formula: `neutral_mass = (m/z × charge) - (charge × 1.0073)`

where 1.0073 Da is the mass of a proton.

<details>
<summary><b>Click to check your answers</b></summary>

**Part A: Charge State**
```
Spacing = 408.26 - 407.76 = 0.50 m/z
Charge = 1 / spacing = 1 / 0.50 = 2
```
The ion has a **+2 charge state**.

**Part B: Neutral Mass**
```python
mz = 407.76
charge = 2
proton_mass = 1.0073

neutral_mass = (mz * charge) - (charge * proton_mass)
neutral_mass = (407.76 * 2) - (2 * 1.0073)
neutral_mass = 815.52 - 2.0146
neutral_mass = 813.5 Da
```

The neutral (uncharged) mass of this peptide is approximately **813.5 Da**.

</details>

---

## 4. Total Ion Current (TIC) chromatogram

Finally, we look at the **Total Ion Current chromatogram (TIC)**.

- The TIC shows the **total amount of ions** detected over time.
- More precisely, for each MS spectrum we sum **all peak intensities** and plot this sum vs. **retention time**.
- It is sometimes stored in the spectra files, but we will calculate it manually.

You can think of the TIC as a kind of **retention-time histogram** of the 2D peak map:

- We collapse the m/z axis by summing over all peaks.
- We keep only the retention time and the summed intensity.

We will:

1. Compute the TIC manually from the MS1 spectra.  
2. Plot TIC vs RT.  
3. Interpret the peaks in the TIC (e.g. they correspond to eluting analytes).

Think about how the TIC is related to the eluting analytes from the chromatography?


In [ ]:

tic_rts = []
tic_intensities = []

for spec in exp:
  if spec.getMSLevel() == 1:
      rt = spec.getRT()
      mzs, ints = spec.get_peaks()

      total_intensity = np.sum(ints)  # sum of all intensities in this spectrum

      tic_rts.append(rt)
      tic_intensities.append(total_intensity)

tic_rts = np.array(tic_rts)
tic_intensities = np.array(tic_intensities)

plt.figure(figsize=(10, 4))
plt.plot(tic_rts / 60.0, tic_intensities)
plt.xlabel("Retention time (minutes)")
plt.ylabel("Total ion current (sum of intensities)")
plt.title("Total Ion Current (TIC) chromatogram")
plt.tight_layout()
plt.show()


The TIC peaks indicate time regions where many ions (and thus analytes) elute.
Broad peaks can correspond to complex mixtures, sharp peaks to single analytes.

## Summary

Congratulations! You've completed the first notebook. You learned:

| Topic | Key Concept | pyOpenMS Tools |
|-------|-------------|----------------|
| **Protein digestion** | Trypsin cleaves after K/R (not before P) | `ProteaseDigestion`, `AASequence` |
| **MS data loading** | mzML files contain spectra with m/z and intensity | `MSExperiment`, `MzMLFile` |
| **2D visualization** | Peak maps show RT vs m/z with intensity as color | `get_peaks()`, matplotlib |
| **Isotope patterns** | Spacing = 1/charge; used to determine charge state | Interactive zoom |
| **TIC chromatogram** | Sum of intensities over time shows elution profile | `np.sum()` on peaks |

---

## Bonus Challenges

Test your understanding with these optional challenges:

<details>
<summary><b>Challenge 1 (Beginner): Try a Different Enzyme</b></summary>

Change the enzyme from Trypsin to another protease. Try:
- `"Chymotrypsin"` - cleaves after F, W, Y
- `"Lys-C"` - cleaves only after K
- `"Asp-N"` - cleaves before D

```python
digestion.setEnzyme("Chymotrypsin")  # Try this!
```

**Questions:**
1. How does the number of peptides change?
2. How does the average peptide length change?
3. Why might you choose one enzyme over another?

</details>

<details>
<summary><b>Challenge 2 (Intermediate): Find Another Isotope Pattern</b></summary>

Explore the spectrum to find a +3 charged ion:

1. Look for isotope peaks spaced ~0.33 m/z apart
2. Calculate the neutral mass
3. Compare: would this peptide be longer or shorter than our +2 example?

**Hint**: Higher charge states are often seen for larger peptides because they have more basic residues (K, R, H) to accept protons.

</details>

<details>
<summary><b>Challenge 3 (Advanced): Calculate Base Peak Chromatogram</b></summary>

Instead of TIC (sum of all intensities), calculate the **Base Peak Chromatogram (BPC)**:
- For each spectrum, record only the **maximum intensity peak**
- Plot max_intensity vs RT

```python
# Your code here - modify the TIC code
bpc_rts = []
bpc_intensities = []

for spec in exp:
    if spec.getMSLevel() == 1:
        rt = spec.getRT()
        mzs, ints = spec.get_peaks()
        max_intensity = ???  # What NumPy function gives the maximum?
        bpc_rts.append(rt)
        bpc_intensities.append(max_intensity)
```

**Question**: How does the BPC compare to the TIC? When might you prefer one over the other?

</details>

---

**Next up: [Notebook 2 - Peptide Identification](EUBIC_Task2_ID.ipynb)** - Learn how to identify which peptides produced the spectra you just visualized!